# Research Start

In [1]:
from datetime import datetime
from scipy import stats
import backtrader as bt
# import yfinance_ez as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
import os
import quandl
import statistics

%matplotlib inline
quandl.read_key()

### Creating Futures Time Series

In [ ]:
# contract_details = pd.read_csv('/home/nikki/downloads/continuous.csv')

In [ ]:
# contract_details[contract_details['Quandl Code'] == 'CHRIS/EUREX_FDAX']

In [ ]:
# NQ = quandl.get("CHRIS/CME_NQ1")
# UL = quandl.get("CHRIS/CME_UL1")
# B6 = quandl.get("CHRIS/CME_BP1")
# GC = quandl.get("CHRIS/CME_GC1")
# FDAX = quandl.get("CHRIS/EUREX_FDAX1")
# FGBL = quandl.get("CHRIS/EUREX_FGBL1")
# A6 = quandl.get("CHRIS/CME_AD1")
# HE = quandl.get("CHRIS/CME_LN1")

In [2]:
"""
Current portfolio as of September 2020
8 instruments
Data currently uses continuous non-adjusted futures provided free from Quandle

TODO:
Look for ways to use back-adjusted futures data
"""
contracts = ['CHRIS/CME_NQ', 'CHRIS/CME_UL', 'CHRIS/CME_BP', 'CHRIS/CME_GC', 'CHRIS/EUREX_FDAX', 'CHRIS/EUREX_FGBL', 'CHRIS/CME_AD', 'CHRIS/CME_LN']

In [3]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
data_futures = {}
for contract in contracts:
    data_futures[contract.lower().split('/')[-1]] = quandl.get(f'{contract}1')

In [46]:
data_futures['barchart_fdax'] = pd.read_csv('data/barchart_fdax.csv', index_col=0, parse_dates=True)
data_futures['barchart_fgbl'] = pd.read_csv('data/barchart_fgbl.csv', index_col=0, parse_dates=True)

## Rules Logic

In [4]:
def volatility(ts, lookback=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(lookback).std().iloc[-1]

    # alternative caltulation using ewma
#     return ts.pct_change().ewm(span=lookback).std().iloc[-1]

In [30]:
mac_scaling = {2: 180.8,
               4: 124.32,
               8: 83.84,
               16: 57.12,
               32: 38.24,
               64: 25.28}

ewmac_scaling = {2: 10.6,
                 4: 7.5,
                 8: 5.3,
                 16: 3.75,
                 32: 2.65,
                 64: 1.87}

breakout_scaling = {10: 28.6,
                    20: 31.6,
                    40: 32.7,
                    80: 33.5,
                    160: 33.5,
                    320: 33.5}

vol_lookback = 24

forecast_table = pd.DataFrame(columns=['contract', 'rule', 'forecast', 'ins_risk_pct'])

for contract, timeseries_full in data_futures.items():
    timeseries = timeseries_full['Settle']
    ins_risk_pct = volatility(timeseries, vol_lookback) * 16
    ins_risk_pts = ins_risk_pct * timeseries.iloc[-1]
        
    # MAC forecasts
    for lookback, scaling in mac_scaling.items():
        lookback_fast = lookback
        lookback_slow = 4 * lookback_fast
        ma_fast = timeseries.rolling(lookback_fast).mean().iloc[-1]
        ma_slow = timeseries.rolling(lookback_slow).mean().iloc[-1]
        
        mac_raw = ma_fast - ma_slow
        mac_risk = mac_raw / ins_risk_pts
        mac_scaled = min(20, max(mac_risk * scaling, -20))
        
        forecast_table = forecast_table.append({'contract': contract,
                                                'rule': f'mac_{lookback_fast}_{lookback_slow}',
                                                'forecast': mac_scaled,
                                                'ins_risk_pct': ins_risk_pct},
                                               ignore_index=True)
        
    # EWMAC forecasts    
#     for lookback, scaling in ewmac_scaling.items():
#         lookback_fast = lookback
#         lookback_slow = 4 * lookback_fast
#         ewma_fast = timeseries.ewm(span=lookback_fast).mean().iloc[-1]
#         ewma_slow = timeseries.ewm(span=lookback_slow).mean().iloc[-1]
        
#         ewmac_raw = ewma_fast - ewma_slow
#         ewmac_risk = ewmac_raw / ins_risk_pts
#         ewmac_scaled = min(20, max(ewmac_risk * scaling, -20))
        
#         forecast_table = forecast_table.append({'contract': contract,
#                                                 'rule': f'ewmac_{lookback_fast}_{lookback_slow}',
#                                                 'forecast': ewmac_scaled,
#                                                 'ins_risk_pct': ins_risk_pct},
#                                                ignore_index=True)
        
    # Breakout forecasts
    for lookback, scaling in breakout_scaling.items():
        rolling_max = timeseries.rolling(lookback).max().iloc[-1]
        rolling_min = timeseries.rolling(lookback).min().iloc[-1]
        rolling_avg = (rolling_max + rolling_min) / 2
        breakout_raw = (timeseries.iloc[-1] - rolling_avg) / (rolling_max - rolling_min)
        breakout_scaled = min(20, max(breakout_raw * scaling, -20))
        
        forecast_table = forecast_table.append({'contract': contract,
                                                'rule': f'breakout_{lookback}',
                                                'forecast': breakout_scaled,
                                                'ins_risk_pct': ins_risk_pct},
                                               ignore_index=True)

In [32]:
forecast_table.to_clipboard()

In [7]:
forecast_table

,contract,rule,forecast,ins_risk_pct
0,cme_nq,mac_2_8,-15.063057,0.314722
1,cme_nq,mac_4_16,-16.127744,0.314722
2,cme_nq,mac_8_32,1.280213,0.314722
3,cme_nq,mac_16_64,12.580285,0.314722
4,cme_nq,mac_32_128,17.416181,0.314722
...,...,...,...,...
91,cme_ln,breakout_20,11.732673,0.400926
92,cme_ln,breakout_40,12.626496,0.400926
93,cme_ln,breakout_80,13.849889,0.400926
94,cme_ln,breakout_160,12.306394,0.400926


In [52]:
for key, item in data_futures.items():
    print(key, item.index[-1])

cme_nq 2020-09-14 00:00:00
cme_ul 2020-09-14 00:00:00
cme_bp 2020-09-14 00:00:00
cme_gc 2020-09-14 00:00:00
eurex_fdax 2020-09-11 00:00:00
eurex_fgbl 2020-09-10 00:00:00
cme_ad 2020-09-14 00:00:00
cme_ln 2020-09-14 00:00:00
eurex_fdax2 2020-06-19 00:00:00
eurex_fgbl2 2020-09-11 00:00:00
barchart_fdax 2020-09-14 00:00:00
barchart_fgbl 2020-09-14 00:00:00
